In [85]:
import random
from collections import defaultdict

import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate, train_test_split

 ### Use sql JOIN to get desired columns

```sql
    SELECT p.product_id, o.order_dow, o.user_id, op.add_to_cart_order FROM products p JOIN order_products op ON p.product_id = op.product_id JOIN orders o ON op.order_id = o.order_id;
```
### Ran in in terminal to create csv file for required columns:

```sql
    mysql -u root -p warehouse2 -e "SELECT p.product_id, o.order_dow, o.user_id, op.add_to_cart_order FROM products p JOIN order_products op ON p.product_id = op.product_id JOIN orders o ON op.order_id = o.order_id INTO OUTFILE '~/resultgg.csv' FIELDS TERMINATED BY ',' ENCLOSED BY '\"' LINES TERMINATED BY '\n';"
```

#### Since, we forgot to put the column names while joining the tables, let's do that now!

In [86]:
df = pd.read_csv('./data/final_gg.csv', names=['product_name', 'product_id', 'order_dow', 'user_id', 'add_to_cart_order'])
df.head()

,product_name,product_id,order_dow,user_id,add_to_cart_order
0,Cool Mint Chocolate Energy Bar,5531,3,90,10
1,"Healthy Grains Granola Bar, Vanilla Blueberry",1545,3,90,7
2,Peach-Pear Sparkling Water,1831,3,90,4
3,Total 2% Lowfat Greek Strained Yogurt With Blu...,4957,2,451,3
4,Organic Cream Of Chicken Condensed Soup,5499,0,503,1


#### Top Banana Buyers
- Get the top 5 user_ids from users who frequently buy bananas
- Case match is false
- Handled edge case of finding both bananas and banana in product name
- [62443, 69919, 151361, 120232, 194931] These are the top users according to dataset, just putting here for reference so that we can compare later or unit test it


In [87]:
df_atc = df[df['add_to_cart_order'] < 5]
bananas_df = df_atc[df_atc['product_name'].str.contains("Bananas|Banana", case=False)]
print(bananas_df.shape)
top_banana_buyers = bananas_df['user_id'].value_counts().index.tolist()[:5]

top_banana_buyers

(324645, 5)


[62443, 69919, 151361, 120232, 194931]

> The ratings are used in collaborative filtering as if two people like the same thing once, they are likely to have similar taste for other items as well. Well, in our case, we have add_to_cart_order. means how much they like them.

> Ratings are also used in matrix factorization as the matrix cells values determining which user liked which product how much?

##### Since, ratings are the building blocks for collaborative filtering and matrix factorization, and we do NOT have rating values, we will use "add_to_cart_order" as the rating. It makes sense because how many time user has added a product in cart ideally leads to how much they like them.

In [88]:
# Surprise Prep 
reader = Reader(rating_scale=(1, df_atc['add_to_cart_order'].max()))
data = Dataset.load_from_df(df_atc[['user_id', 'product_id', 'add_to_cart_order']], reader)

# Train Prep
train, test = train_test_split(data, test_size=0.2)
model = SVD()

##### Since, we need to consider strong and weak generalizations, let's prepare those now!

> We know that strong generalizations are just the testing sets, but the weak generalization are any items that are in the training set. So, for weak generalization we will take 1000 random entries from the data and then use **(the user and user_dow combination)** to generate a weak generalized testing set

> order_dow is the order day of week, 0=Monday, 1=Tuesday so on... (at least this is our assumption here)

In [89]:
df_user_dow_uniques = df_atc[['user_id', 'order_dow']].drop_duplicates()
# df_user_dow_randoms = df_user_dow_uniques.sample(1000, random_state=42, replace=False)
df_user_dow_randoms = df_user_dow_uniques.sample(1000, replace=False) # Using replace to delete the value after taking it, to ensure uniqueness

strong_test = list()
for _, row in df_user_dow_randoms.iterrows():
    user = row['user_id']
    order_dow = row['order_dow']
    # print("GG: ", row['order_dow'])
    user_dow_products = df_atc[(df_atc['user_id'] == user) & (df_atc['order_dow'] == order_dow)]
    # print("EZ: ", user_dow_products)
    for _, product_row in user_dow_products.iterrows():
        strong_test.append((user, product_row['product_id'], product_row['add_to_cart_order']))
print(len(strong_test))

7240


### Optimization of time for the loop by using **Indexing:** Pretty fast: :) [Runtime decreased from 7.6 sec to 0.8 sec] (Tested with random_state=42)
> We create an indexed dataset, a new data frame because the columns will be used as index and will not be available for use after we do this. So, we will stored the indexed values in new data frame. 

> Later we can use this to get the combination of user and user_dow :

```python
    products = indexed_df_atc.loc[(user, order_dow)]
```

In [90]:
indexed_df_atc = df_atc.set_index(['user_id', 'order_dow'])
indexed_df_atc.sort_index(inplace=True)

In [91]:
df_user_dow_uniques = df_atc[['user_id', 'order_dow']].drop_duplicates()
# df_user_dow_randoms = df_user_dow_uniques.sample(1000, random_state=42, replace=False)
df_user_dow_randoms = df_user_dow_uniques.sample(1000, replace=False)

strong_test = list()
for _, row in df_user_dow_randoms.iterrows():
    user = row['user_id']
    order_dow = row['order_dow']
    # print("GG: ", row['order_dow'])
    user_dow_products = indexed_df_atc.loc[(user, order_dow)]
    # print("EZ: ", user_dow_products)
    for _, product_row in user_dow_products.iterrows():
        strong_test.append((user, product_row['product_id'], product_row['add_to_cart_order']))
print(len(strong_test))

7380


#### Fitting in t_minus.... secs

In [92]:
model.fit(train)
predictions = model.test(test)

## Predictions for top 5 Banana buyers

In [93]:
for user in top_banana_buyers:
    df_top_buyers = df_atc[df_atc['user_id'] == user]
    results = []
    for _, row in df_top_buyers.iterrows():
        pred = model.predict(user, row['product_id'])
        results.append((row['product_name'], pred.est))
    results.sort(key=lambda x: x[1], reverse=True)
    print(f"Top recommendation for user {user}: {results[0][0]}")

Top recommendation for user 62443: Peach,  Apricot & Banana Stage 2 Baby Food
Top recommendation for user 69919: Plus Lotion Facial Tissues
Top recommendation for user 151361: Organic Baby Arugula
Top recommendation for user 120232: Organic Gala Apples
Top recommendation for user 194931: Fat Free Milk


#### Prediction object from surprise contains user_id, product_id, true_atc, est_atc, details. We will take them and create a new dictionary of list of tuples where inner tuples contains the (product_id, true_atc, est_atc), then we sort it by the 3rd value of tuple which is estimated_atco. After that we can create two lists containing the actual products and the predicted products for each users in the dictionary.

In [94]:
users_with_top_n_predictions = dict()

for user_id, product_id, true_atc, est_atc, _ in predictions:
    # Need to check if user is already in the dict key, if is not then need to create new entry for user in dict, else add predictions(product_id, true_atc, est_atc) to the same user.
    if user_id in users_with_top_n_predictions:
        users_with_top_n_predictions[user_id].append((product_id, true_atc, est_atc))
    else:
        users_with_top_n_predictions[user_id] = [(product_id, true_atc, est_atc)]

# users_with_top_n_predictions

#### The value of thresh is the threshold for atco, if it is greater than mean, the relevancy of the recommended product is high. So, let's put is a little bit greater than mean which is 2 (ratings being 0 to 4 as the atco < 5)
> We will experiment with thresh later

In [95]:
k = 5
thresh = 3
actual, predicted = list(), list()
for user_id, product_true_est in users_with_top_n_predictions.items():
    product_true_est.sort(key=lambda x: x[2], reverse=True) # sorting by 3rd value (estimated atc) of tuple inside the list inside that dict (users_with_top_n_predictions)
    actual.append([product_id for (product_id, true_atc, _) in product_true_est if true_atc >= thresh])
    # Because the product_true_est object is sorted according to estimated atc(or estimated rating), we can take the top k elements for predicted
    predicted.append([product_id for (product_id, _, _) in product_true_est[:k]])

print(actual[:7])
print(predicted[:7])
    

[[45441, 45441, 6907, 40997, 39170], [19678, 19678, 32293, 4799, 43961, 21938, 30233, 27104, 21137, 21137, 47209, 47209, 47209, 5618, 12442, 15872, 20119, 32655], [25718, 25718, 21463, 46667, 4461, 41072, 39409, 39577, 31973, 32652, 32652, 32652, 32652, 32652, 21903, 38311, 38311, 12341, 21174, 11109, 4472, 4472, 4472], [11365, 33000, 30406], [25691, 36189, 4697, 2275], [42014, 32691], [8193, 8193, 44133, 7021, 23270, 23270, 23270, 23270, 2295, 37107, 21903, 2979, 5120, 5120, 5120, 24852, 37220]]
[[4155, 45441, 45441, 6907, 38374], [19678, 19678, 32293, 32293, 4799], [25718, 25718, 25718, 2078, 21463], [11365, 33000, 30406, 30406, 5068], [25691, 36189, 36189, 4697, 2275], [43139, 4313, 42014, 32691], [8193, 8193, 38273, 44133, 7021]]


#### To calculate MAP@5, need to create function. GG, will do later... TODO
#### Once the function is ready, we already have weak and strong generalizations created to test